In [1]:
import itertools
import pandas as pd
from tqdm.notebook import tqdm
%run keras_sentinel.ipynb

2024-05-16 14:19:45.100430: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 14:19:45.969960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_dir = Path('models').joinpath('bands')
model_dir.mkdir(parents=True, exist_ok=True)

years = [2017, 2018, 2019]
all_seasons = [3, 6, 9, 12]
all_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

band_combinations = itertools.chain.from_iterable(
    itertools.combinations(all_bands, r) for r in range(1, len(all_bands)+1))
band_combinations = list(band_combinations)

In [ ]:
filepaths = []
for year in years:
    for season in season_combination:
        file_key = f'processed_treesat_{year}*{season}.npy'
        filepaths.extend(sorted(list(Path('seasonal_median').glob(file_key))))
        
selected_data = []
for filepath in filepaths:
    with open(filepath, 'rb') as f:
        data = np.load(f)
    selected_data.append(data)
features = np.stack(selected_data, axis=3)

results = []
for band_combination in band_combinations:
    model_name = f'conv_seasons_all_years_{"_".join(map(str, years))}_bands_{"-".join(map(str, band_combination))}.keras'
    model_eval = Conv3DModelCreator().run(labels, features, model_name, model_dir, epochs=3, overwrite=False)
    results.append(model_eval)

In [3]:
results_df = pd.DataFrame(results)
results_df.insert(0, 'season_comb', season_combinations)
results_df

,season_comb,f1_majority,loss,r2_score,root_mean_squared_error
0,"(3,)",0.503697,0.017613,0.217599,0.181754
1,"(6,)",0.519723,0.017746,0.221932,0.182205
2,"(9,)",0.455417,0.018966,0.182390,0.189085
3,"(12,)",0.506661,0.017218,0.238422,0.179486
4,"(3, 6)",0.575742,0.015402,0.310635,0.165755
5,"(3, 9)",0.546298,0.018522,0.181324,0.184073
6,"(3, 12)",0.591104,0.016694,0.286856,0.173443
7,"(6, 9)",0.492239,0.020294,0.170416,0.193066
8,"(6, 12)",0.590116,0.016068,0.297170,0.170413
9,"(9, 12)",0.553718,0.017574,0.217348,0.178878


In [4]:
# major_pred = y_pred.argmax(axis=1)
# major_true = y_true.argmax(axis=1)

# cr = classification_report(
#     major_true, major_pred, target_names=df[target].columns, 
#     output_dict=True, zero_division=np.nan
# )
# df_report = pd.DataFrame(cr).T
# df_report['support'] = df_report['support'].astype(int)

# display(HTML(df_report.round(decimals=2).to_html()))